In [1]:
# 여기에 코드를 작성하세요.
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
import gradio as gr

propmt = ChatPromptTemplate.from_messages([
    ("system","당신은 여행 일정 계획을 도와주는 여행 컨설턴트 입니다. 날짜, 장소, 예산 등 필요한 정보들을 순서대로 물어봐주세요. 한번에 하나씩 물어봐주세요"),
    MessagesPlaceholder("chat_history"),
    ("system", "이전 대화를 참고하여서 일정을 계획해주세요."),
    ("human","{user_input}")
])

model = ChatOpenAI(
    model="gpt-4.1-nano",
    temperature=0.3,
    top_p=0.9
)

chain = propmt | model | StrOutputParser()

def answer_invoke(message, history, travel_period, travel_region, travel_style, travel_budget):
    if not travel_style:
        style_text = "미정"
    elif isinstance(travel_style, list):
        style_text = ", ".join(travel_style)
    else:
        style_text = str(travel_style)

    history_messages = [
        HumanMessage(content=f"여행 기간은 {travel_period}입니다."),
        HumanMessage(content=f"여행 지역은 {travel_region}입니다."),
        HumanMessage(content=f"여행 스타일은 {style_text}입니다."),
        HumanMessage(content=f"예산은 {travel_budget}입니다.")
    ]

    if not message and not history:
        message = (
            "지금까지 정보를 바탕으로, "
            "다음 여행 일정을 짜기 위해 가장 먼저 필요한 질문을 하나만 해주세요."
        )


    for msg in history:
        if msg['role'] == "user":
            history_messages.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_messages.append(AIMessage(content=msg['content']))

    response = chain.invoke({
        "chat_history": history_messages,
        "user_input": message
    })
    return response
        
with gr.Blocks() as demo:
    travel_period = gr.State("")
    travel_region = gr.State("")
    travel_style = gr.State([])
    travel_budget = gr.State("")

    with gr.Group(visible=True) as step0:
        gr.Markdown("## 📅 여행 기간을 입력해주세요 (YYYY-MM-DD)")

        start_date = gr.Textbox(label="여행 시작일", placeholder="2026-03-01")
        end_date = gr.Textbox(label="여행 종료일", placeholder="2026-03-04")

        step0_next = gr.Button("다음 ▶")

    with gr.Group(visible=False) as step1:
        gr.Markdown("## 여행 지역을 선택해주세요")

        region = gr.Radio(["국내", "해외"],label="여행 지역")
        step1_next=gr.Button("다음 ▶")

    with gr.Group(visible=False) as step2:
        gr.Markdown("## 🎒 여행 스타일을 선택해주세요 (복수 선택 가능)")

        style = gr.CheckboxGroup(
            ["휴식", "관광", "미식", "자연", "사진", "쇼핑"],
            label="여행 스타일"
        )

        step2_next = gr.Button("다음 ▶")

    with gr.Group(visible=False) as step3:
        gr.Markdown("## 💰 여행 예산 범위를 선택해주세요")

        budget = gr.Radio(
            [
                "가성비 (50만원 이하)",
                "보통 (50~100만원)",
                "여유 (100~200만원)",
                "럭셔리 (200만원 이상)"
            ],
            label="1인 기준 예산"
        )

        step3_next = gr.Button("다음 ▶")    


    # Chat 영역 (처음엔 숨김)
    with gr.Group(visible=False) as chat_group:
        chat = gr.ChatInterface(
            fn=answer_invoke,
            additional_inputs=[
                travel_period,
                travel_region,
                travel_style,
                travel_budget
            ],
            title="✈️ 여행 일정 어시스턴트"
        )
    
    # STEP 0 → STEP 1 (여행 기간 → 국내/해외)
    def go_to_step1(start, end):
        period = f"{start} ~ {end}"
        return (
            period,                     # travel_period 저장
            gr.update(visible=False),   # step0 숨김
            gr.update(visible=True)     # step1 표시
        )

    step0_next.click(
        go_to_step1,
        inputs=[start_date, end_date],
        outputs=[travel_period, step0, step1]
    )


    # STEP 1 → STEP 2 (국내/해외 → 여행 스타일)
    def go_to_step2(region):
        return (
            region,                     # travel_region 저장
            gr.update(visible=False),   # step1 숨김
            gr.update(visible=True)     # step2 표시
        )
    
    def go_to_step3(style):
        return (
            style,                      # travel_style 저장
            gr.update(visible=False),   # step2 숨김
            gr.update(visible=True)     # step3 표시
        )

    step1_next.click(
        go_to_step2,
        inputs=[region],
        outputs=[travel_region, step1, step2]
    )

    step2_next.click(
        go_to_step3,
        inputs=[style],
        outputs=[travel_style, step2, step3]
    )

    # STEP 2 → CHAT (여행 스타일 → 대화 시작)
    def go_to_chat(budget):
        return (
            budget,                     # travel_budget 저장
            gr.update(visible=False),   # step3 숨김
            gr.update(visible=True)     # chat_group 표시
        )

    step3_next.click(
        go_to_chat,
        inputs=[budget],
        outputs=[travel_budget, step3, chat_group]
    )

demo.launch()

c:\project\llm_edu\001_chatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


In [ ]:
# Gradio 인터페이스 종료
demo.close()